# **CCC-BIDMC Schedule Generator**

**Semester:** Summer 2024

**Ayush Noori** <br>
**Crimson Care Collaborative Clinic** <br>
**May 19, 2024**

---

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Path management
from pathlib import Path

# Get downloads folder
# data_dir = Path.home() / "Downloads"
data_dir = Path.cwd() / "data" / "summer_2024"
output_dir = Path.cwd() / "output" / "summer_2024"

Read in the availability data.

In [2]:
# Read in form responses
returning_responses = "CCC-BIDMC Summer 2024 Returning Interest Form (Responses) - Form Responses 1.csv"
new_responses = "CCC-BIDMC Summer 2024 New Volunteer Scheduling (Responses) - Form Responses 1.csv"
df = pd.read_csv(data_dir / returning_responses)

# Read new response
df_new = pd.read_csv(data_dir / new_responses)
df["new"] = "No"
df_new["new"] = "Yes"

# Concatenate dataframes
df.columns = ['timestamp', 'first', 'last', 'role', 'return', 'epic', 'shifts', 'comments', 'new']
df_new.columns = ['timestamp', 'first', 'last', 'role', 'return', 'epic', 'shifts', 'comments', 'new']
df = pd.concat([df, df_new], ignore_index=True)

# Set column names
df["first"] = df["first"].str.strip()
df["last"] = df["last"].str.strip()

# Convert time stamp to datetime
df["timestamp"] = pd.to_datetime(df["timestamp"])

# Take the most recent response
df = df.sort_values("timestamp").groupby(["first", "last"]).last().reset_index()
df = df[df["return"] == "Yes"]
df.reset_index(drop=True, inplace=True)

Add HEART observers.

In [3]:
# Read HEART contact info
heart_volunteers = pd.read_csv(data_dir / 'heart_volunteers.csv', names=["name", "email"])
heart_availability = pd.read_csv(data_dir / 'heart_availability.csv')
heart_data = []

# Add HEART volunteers to table
for i, row in heart_volunteers.iterrows():
    first_name = row['name'].split(" ")[0]
    last_name = row['name'].split(" ")[1]

    # Get availabilities
    availability = []
    for col in heart_availability:
        if first_name in heart_availability[col].values:
            availability.append(col.split(" BIDMC")[0])

    # Add to data frame
    heart_data.append({
        "first": first_name,
        "last": last_name,
        "role": "Observer",
        "return": "No",
        "epic": "No",
        "shifts": ", ".join(availability)
    })

heart_data = pd.DataFrame(heart_data)
df = pd.concat([df, heart_data], ignore_index=True)
df = df.where(pd.notnull(df), None)

Generate availability table.

In [4]:
# Generate summer clinic dates
start_date = "June 4, 2024"
end_date = "September 17, 2024"

# Weekly dates on Tuesdays
dates = pd.date_range(start=start_date, end=end_date, freq="W-TUE")
dates = dates.strftime("%-m/%-d/%Y").values

# Create a dataframe with name, role, and dates
availability = pd.DataFrame(columns=["Name", "Role"] + list(dates))

Populate availability table with form responses.

In [5]:
# Iterate over the responses
for i, row in df.iterrows():
    
    # Get the name and role
    name = f"{row['first']} {row['last']}"
    if row['new'] == "Yes":
        name = f"{row['first']} {row['last']}"
    role = row["role"]
    
    # Get the shifts
    shifts = row["shifts"].split(", ")
    year = 2024
    shifts = [pd.to_datetime(f"{shift} {year}").strftime("%-m/%-d/%Y") for shift in shifts]
    
    # Create a row
    schedule = [name, role] + ["" if date in shifts else "x" for date in dates]
    
    # Add the row to the dataframe
    availability.loc[i] = schedule

Mark all new volunteers as unavailable for the first two weeks.

In [6]:
# Get all new volunteers
new_volunteers = df[(df["new"] == "Yes") & (df["role"] != "Observer")]

# Set all new volunteer availability to "x" for first two weeks
for i, row in new_volunteers.iterrows():
    
    # Get the name
    name = f"{row['first']} {row['last']}"
    print("Updating availability for", name)

    # Update availability for first two weeks
    availability.loc[availability["Name"] == name, dates[:2]] = "x"

Updating availability for Akua Pokuaa
Updating availability for Amanda Law
Updating availability for Caroline Diggins
Updating availability for Christopher Robinson
Updating availability for Corinne Auger
Updating availability for Denny Lu
Updating availability for Ege Eskibozkurt
Updating availability for Emily Yang
Updating availability for Esmé Wheeler
Updating availability for Fielding Fischer
Updating availability for Kevin Cho
Updating availability for Massoud Sharif
Updating availability for Michael Chen
Updating availability for Nishmi Abeyweera
Updating availability for Romila Santra
Updating availability for Sojas Wagle
Updating availability for Xavier Vasquez


Add PCC (Senior Clinicians) every two weeks, and grant them highest scheduling priority. Finally, add back-up junior and senior clinicians.

In [7]:
# Add PCC 1-4 to the availability
pcc_clinicans = ["PCC 1", "PCC 2", "PCC 3", "PCC 4"]
for pcc in pcc_clinicans:

    # PCC start and end dates
    pcc_start_1 = "June 4, 2024"
    pcc_end_1 = "June 18, 2024"
    pcc_start_2 = "July 9, 2024"
    pcc_end_2 = "September 17, 2024"

    # Make availability every other week
    pcc_dates_1 = pd.date_range(start=pcc_start_1, end=pcc_end_1, freq="2W-TUE")
    pcc_dates_1 = pcc_dates_1.strftime("%-m/%-d/%Y").values
    pcc_dates_2 = pd.date_range(start=pcc_start_2, end=pcc_end_2, freq="2W-TUE")
    pcc_dates_2 = pcc_dates_2.strftime("%-m/%-d/%Y").values
    pcc_dates = np.concatenate([pcc_dates_1, pcc_dates_2])
    pcc_avail = ["" if date in pcc_dates else "x" for date in dates]

    # Add a row for each PCC
    availability.loc[len(availability)] = [pcc, "Senior Clinician"] + pcc_avail
    print("Adding availability for", pcc)

Adding availability for PCC 1
Adding availability for PCC 2
Adding availability for PCC 3
Adding availability for PCC 4


Process and save availability table.

In [8]:
# Order by role
role_order = ["Senior Clinician", "Junior Clinician", "Senior Director", "Administrative Manager", "Observer"]
availability["Role"] = pd.Categorical(availability["Role"], categories=role_order, ordered=True)
availability = availability.sort_values(["Role", "Name"]).reset_index(drop=True)

Schedule clinic shifts based on availability and even distribution of shifts.

In [9]:
# First, schedule everyone
schedule = availability.copy()
schedule = schedule.replace("", "s")

# Maximum shifts by role
max_shifts = {
    "Senior Clinician": 6,
    "Junior Clinician": 6,
    "Senior Director": 1,
    "Administrative Manager": 1,
    "Observer": 3
} 

# Create counter of shifts per person
schedule["Counts"] = schedule[dates].apply(lambda x: x.value_counts()["s"], axis=1)
schedule["Counts"] = schedule["Counts"].astype(float)

# Grant PCC highest priority by setting counter to 0
schedule.loc[schedule["Name"].str.contains("PCC"), "Counts"] = 0

Generate schedule table.

In [10]:
# Iterate over the dates
for date in dates:

    # Iterate over the roles
    for role in max_shifts.keys():

        # Get the people who are scheduled
        scheduled = schedule[schedule[date] == "s"]
        scheduled = scheduled[scheduled["Role"] == role]

        # If the number of people is more than the maximum
        if len(scheduled) > max_shifts[role]:

            # Sort greatest to least, since we subtract from counts
            scheduled = scheduled.sort_values("Counts", ascending=False)

            # Drop people with the fewest shifts until the number is correct
            while len(scheduled) > max_shifts[role]:

                person = scheduled.iloc[0]
                schedule.loc[schedule["Name"] == person["Name"], date] = ""
                schedule.loc[schedule["Name"] == person["Name"], "Counts"] -= 1
                scheduled = scheduled.drop(person.name)

            # For SCs and JCs, schedule one more backup if available
            if role in ["Senior Clinician", "Junior Clinician"]:

                available = schedule[schedule[date] == ""]
                available = available[available["Role"] == role]

                # If there are people available, schedule one more as backup
                if len(available) > 0:

                    # Sort least to greatest, since we add back to counts
                    available = available.sort_values("Counts", ascending=True)

                    person = available.iloc[0]
                    schedule.loc[schedule["Name"] == person["Name"], date] = "b"
                    schedule.loc[schedule["Name"] == person["Name"], "Counts"] += 0.5
                    print(f"Scheduling {person['Name']} as backup for {date}")

Scheduling Kathryn Li as backup for 6/18/2024
Scheduling Anika Park as backup for 6/18/2024
Scheduling Anika Park as backup for 6/25/2024
Scheduling Massoud Sharif as backup for 7/2/2024
Scheduling Ege Eskibozkurt as backup for 7/9/2024
Scheduling Anika Park as backup for 7/9/2024
Scheduling Akshay Jaggi as backup for 7/16/2024
Scheduling Anika Park as backup for 7/16/2024
Scheduling Akshay Jaggi as backup for 7/23/2024
Scheduling Anika Park as backup for 7/23/2024
Scheduling Karina Aguilar as backup for 7/30/2024
Scheduling Ege Eskibozkurt as backup for 8/6/2024
Scheduling Amanda Law as backup for 8/6/2024
Scheduling Justin Gelman as backup for 8/13/2024
Scheduling Corinne Auger as backup for 8/13/2024
Scheduling Kathryn Li as backup for 8/20/2024
Scheduling Kaiz Esmail as backup for 8/20/2024
Scheduling Caroline Diggins as backup for 8/27/2024
Scheduling Michael Chen as backup for 8/27/2024
Scheduling Debby Cheng as backup for 9/3/2024
Scheduling Amanda Law as backup for 9/3/2024
Sch

Save to file.

In [11]:
# Save availability and schedule
availability.to_csv(output_dir / "availability.csv", index=False, encoding="utf-8-sig")
schedule.to_csv(output_dir / "schedule.csv", index=False, encoding="utf-8-sig")

Create master schedule table.

In [12]:
# Create master schedule document
# Columns: 
# A: Senior Clinician 1
# A: Junior Clinician 1
# A: Senior Clinician 2
# A: Junior Clinician 2
# A: Senior Clinician 3
# A: Junior Clinician 3
# B: Senior Clinician 1
# B: Junior Clinician 1
# B: Senior Clinician 2
# B: Junior Clinician 2
# B: Senior Clinician 3
# B: Junior Clinician 3
# Senior Director

# Backup Senior Clinician
# Backup Junior Clinician
# Administrative Manager

# Observer 1
# Observer 2
# Observer 3

# Create a new dataframe for the master schedule
master_schedule = pd.DataFrame(columns=[
    "A: Senior Clinician 1", "A: Junior Clinician 1",
    "A: Senior Clinician 2", "A: Junior Clinician 2",
    "A: Senior Clinician 3", "A: Junior Clinician 3",
    "B: Senior Clinician 1", "B: Junior Clinician 1",
    "B: Senior Clinician 2", "B: Junior Clinician 2",
    "B: Senior Clinician 3", "B: Junior Clinician 3",
    "Senior Director", "Backup Senior Clinician",
    "Backup Junior Clinician", "Administrative Manager",
    "Observer 1", "Observer 2", "Observer 3"])

# Iterate over the dates
for date in dates:

    # Get the people scheduled for the date
    scheduled = schedule[schedule[date] == "s"]

    # Get the people scheduled as backup
    backup = schedule[schedule[date] == "b"]

    # Get the senior clinicians
    scs = scheduled[scheduled["Role"] == "Senior Clinician"]
    scs = scs.sort_values("Counts", ascending=False)

    # Sort PCCs first
    scs = pd.concat([scs[scs["Name"].str.contains("PCC")], scs[~scs["Name"].str.contains("PCC")]])

    # Get the junior clinicians
    jcs = scheduled[scheduled["Role"] == "Junior Clinician"]
    jcs = jcs.sort_values("Counts", ascending=False)

    # Get the senior directors
    sds = scheduled[scheduled["Role"] == "Senior Director"]
    sds = sds.sort_values("Counts", ascending=False)

    # Get the administrative managers
    ams = scheduled[scheduled["Role"] == "Administrative Manager"]
    ams = ams.sort_values("Counts", ascending=False)

    # Get the backup senior clinicians
    bscs = backup[backup["Role"] == "Senior Clinician"]
    bscs = bscs.sort_values("Counts", ascending=False)

    # Get the backup junior clinicians
    bjcs = backup[backup["Role"] == "Junior Clinician"]
    bjcs = bjcs.sort_values("Counts", ascending=False)

    # Get the backup senior directors
    bsds = backup[backup["Role"] == "Senior Director"]
    bsds = bsds.sort_values("Counts", ascending=False)

    # Get the backup administrative managers
    bams = backup[backup["Role"] == "Administrative Manager"]
    bams = bams.sort_values("Counts", ascending=False)

    # Get the observers
    obs = scheduled[scheduled["Role"] == "Observer"]

    # Add the people to the master schedule
    master_schedule.loc[date] = [
        
        # A
        scs.iloc[0]["Name"] if len(scs) > 0 else "",
        jcs.iloc[0]["Name"] if len(jcs) > 0 else "",

        scs.iloc[1]["Name"] if len(scs) > 1 else "",
        jcs.iloc[1]["Name"] if len(jcs) > 1 else "",
        
        scs.iloc[4]["Name"] if len(scs) > 4 else "",
        jcs.iloc[4]["Name"] if len(jcs) > 4 else "",
        
        # B
        scs.iloc[2]["Name"] if len(scs) > 2 else "",
        jcs.iloc[2]["Name"] if len(jcs) > 2 else "",

        scs.iloc[3]["Name"] if len(scs) > 3 else "",
        jcs.iloc[3]["Name"] if len(jcs) > 3 else "",
        
        scs.iloc[5]["Name"] if len(scs) > 5 else "",
        jcs.iloc[5]["Name"] if len(jcs) > 5 else "",
        
        # Senior Director
        sds.iloc[0]["Name"] if len(sds) > 0 else "",
        bscs.iloc[0]["Name"] if len(bscs) > 0 else "",
        bjcs.iloc[0]["Name"] if len(bjcs) > 0 else "",
        ams.iloc[0]["Name"] if len(ams) > 0 else "",

        # Observers
        obs.iloc[0]["Name"] if len(obs) > 0 else "",
        obs.iloc[1]["Name"] if len(obs) > 1 else "",
        obs.iloc[2]["Name"] if len(obs) > 2 else ""
    ]

# Replace PCC labels with names
pcc_names = {
    "PCC 1": "[PCC] Jaeyoon Cha",
    "PCC 2": "[PCC] Ashish Dahal",
    "PCC 3": "[PCC] Andrew Nguyen",
    "PCC 4": "[PCC] Martin Viola"
}

# Replace PCC labels with names
for pcc, name in pcc_names.items():
    master_schedule = master_schedule.replace(pcc, name)

# Transpose the master schedule
master_schedule = master_schedule.T

# Save the master schedule
master_schedule.to_csv(output_dir / "master_schedule.csv", index=True, encoding="utf-8-sig")

Generate roster.

In [59]:
# Read old roster
prev_roster = pd.read_csv(data_dir / "spring_2024_roster.csv", dtype={"Number": int, "Cell Phone": str})

# Filter for returning volunteers
prev_roster = prev_roster[
    (prev_roster['Role'] == "Attending") |
    (prev_roster['Role'] == "Clinic Operations Director") |
    (prev_roster['First'] + " " + prev_roster['Last']).isin(["Marty Viola"]) |
    (prev_roster['First'] + " " + prev_roster['Last']).isin(availability['Name'])
]

# Add new volunteers
new_roster = pd.read_csv(data_dir / "BIDMC New Volunteers Summer 2024.csv")
new_roster = new_roster[["Unnamed: 0", "Name", "Email", "Phone Number"]]
new_roster.columns = ["Role", "Name", "Email", "Cell Phone"]
new_roster = new_roster.dropna(subset=["Name"])

# Split name into first and last
new_roster["First"] = new_roster["Name"].str.split(" ").str[0]
new_roster["Last"] = new_roster["Name"].str.split(" ").str[1]

# Drop name column
new_roster = new_roster.drop(columns=["Name"])

# Map roles
role_map = {
    'BIDMC (AM)': "Administrative Manager",
    'BIDMC (JC)': "Junior Clinician",
    'BIDMC (SC)': "Senior Clinician",
}
new_roster["Role"] = new_roster["Role"].map(role_map)

# Make email lowercase
new_roster["Email"] = new_roster["Email"].str.lower()

# Add HEART volunteers
heart_volunteers = pd.read_csv(data_dir / 'heart_volunteers.csv', names=["name", "email"])
heart_volunteers["Role"] = "Observer"
heart_volunteers["First"] = heart_volunteers["name"].str.split(" ").str[0]
heart_volunteers["Last"] = heart_volunteers["name"].str.split(" ").str[1]
heart_volunteers = heart_volunteers.drop(columns=["name"])
heart_volunteers["Email"] = heart_volunteers["email"].str.lower()
heart_volunteers = heart_volunteers.drop(columns=["email"])
roster = pd.concat([prev_roster, new_roster, heart_volunteers], ignore_index=True)

# Order by role, then name
role_order = ["Attending", "Clinic Operations Director", "Senior Clinician", "Junior Clinician",
              "Senior Director", "Administrative Manager", "Observer"]
roster["Role"] = pd.Categorical(roster["Role"], categories=role_order, ordered=True)
roster = roster.sort_values(["Role", "First", "Last"]).reset_index(drop=True)

# Set number as within role index
roster["Number"] = roster.groupby("Role", observed=False).cumcount() + 1

# Strip all non-numeric characters from phone number
roster["Cell Phone"] = roster["Cell Phone"].str.replace(r'\D', '', regex=True)

# Format phone number
roster["Cell Phone"] = roster["Cell Phone"].apply(lambda x: f"({x[:3]}) {x[3:6]}-{x[6:]}" if  pd.notnull(x) else None)

# Save to CSV
roster.to_csv(output_dir / "summer_2024_roster.csv", index=False, encoding="utf-8-sig")